In [8]:
!pip install -q streamlit pyngrok


In [4]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [33]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
file_path = "Crop_recommendation.csv"
df = pd.read_csv(file_path)

le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

# Split Features and Target
X = df.drop(columns=["label"])
y = df["label"]


# Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Random Forest with Hyperparameter Tuning
rf = RandomForestClassifier(random_state=42)
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5, 10]
}


grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


# Train the Best Model
best_model.fit(X_train, y_train)
# Load your saved model
with open("crop_model.pkl", "rb") as f:
    model = pickle.load(f)

st.title("🌾 Crop Recommendation System")

# These are your form inputs
N = st.number_input("Nitrogen", min_value=0.0, max_value=140.0)
P = st.number_input("Phosphorus", min_value=0.0, max_value=150.0)
K = st.number_input("Potassium", min_value=0.0, max_value=200.0)
temperature = st.number_input("Temperature (°C)", min_value=0.0, max_value=50.0)
humidity = st.number_input("Humidity (%)", min_value=0.0, max_value=100.0)
ph = st.number_input("Soil pH", min_value=0.0, max_value=14.0)
rainfall = st.number_input("Rainfall (mm)", min_value=0.0, max_value=300.0)

# Predict button
if st.button("Predict"):
    data = [[N, P, K, temperature, humidity, ph, rainfall]]
    prediction = model.predict(data)
    st.success(f"✅ Recommended Crop: {prediction[0]}")


Overwriting app.py


In [35]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "2w7mRCYUI0Aq3WduY7mWN4BmsW7_NGSaEiProdDNimPa17wQ"


In [ ]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.188.236.65:8501



In [38]:

from pyngrok import ngrok, conf

conf.get_default().auth_token = "2w7mRCYUI0Aq3WduY7mWN4BmsW7_NGSaEiProdDNimPa17wQ"
public_url = ngrok.connect(addr="8501")
print(f"Your app is live at: {public_url}")

Your app is live at: NgrokTunnel: "https://6796-35-188-236-65.ngrok-free.app" -> "http://localhost:8501"
